# UK COVID-19 Dashboard

### A comparison of occupied mechanical ventilator beds and new cases with first and second vaccine doses

[Source code](https://github.com/jmdwrntn/ukcovid19dashboard) (C) James Thornton, 2021 ([jmdwrntn@mailbox.org](mailto:jmdwrntn@mailbox.org)). All rights reserved.
 *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*

In [46]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [47]:
# Matplotlib magic function which draws graphs inline within a frontend, in this case in the Jupyter notebook
%matplotlib inline
# Make figures larger
plt.rcParams['figure.dpi'] = 100

In [48]:
# Loads JSON data during dashboard startup, more recent API data is optional
with open("beds.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

In [49]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [50]:
def wrangle_data(rawdata):
    # Takes either JSON or fresh data from API and returns a dataframe
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])   

    index=pd.date_range(startdate, enddate, freq='D')
    df=pd.DataFrame(index=index, columns=['beds', 'cases', 'firstDose', 'secondDose'])

    for entry in datalist: # Each entry is a dictionary with date, beds, cases, and vaccine doses
        parsed_date=parse_date(entry['date'])
        for column in ['beds', 'cases', 'firstDose', 'secondDose']:
        # Check that nothing is there yet - just in case some dates are duplicated
            if pd.isna(df.loc[parsed_date, column]): 
            # Replace None with 0.0 in data
                value=float(entry[column]) if entry[column]!=None else 0.0
            # Access single location with .loc and put index,column in a single set of [ ]
                df.loc[parsed_date, column]=value
            
    # Fill in any remaining gaps due to missing dates
    df.fillna(0.0, inplace=True)

    return df

# Wrangle the existing JSON data and produce a dataframe
df=wrangle_data(jsondata)

In [51]:
# National level statistics
filters = [
    "areaType=overview"
]

# Mechanical beds operated by COVID patients, new COVID cases, new first vaccine dose and new second vaccine dose statistics by date
structure = {
    "date": "date",
    "beds": "covidOccupiedMVBeds",
    "cases": "newCasesByPublishDate",
    "firstDose": "newPeopleVaccinatedFirstDoseByPublishDate",
    "secondDose": "newPeopleVaccinatedSecondDoseByPublishDate"
}

# Poll the API with the above parameters and store in variable
latestdata = Cov19API(filters=filters, structure=structure)

In [52]:
def access_api():
    # Accesses PHE API and returns data in same format as existing JSON data
    return latestdata.get_json()

In [56]:
def api_button_callback(button):
    # Get fresh data from the API
    apidata=access_api()
    
    # If the API call returns data, update dataframe and hide button to prevent repeated polling of the API
    if len(apidata) > 0:
        button.icon="check-circle"
        button.description="Synced"
        button.disabled=True
        global df
        df=wrangle_data(apidata)
        refresh_graph()
    # If the API call does not return data, do not update dataframe and provide feedback to user by updating button
    else:
        button.icon="exclamation-circle"
        button.description="Error"
        button.style="warning"
        button.tooltip="Could not sync latest data, falling back to current data"
        button.disabled=True
   
apibutton=wdg.Button(
    description='Sync data',
    disabled=False,
    button_style='',
    tooltip="Sync latest data from PHE",
    icon='cloud-download'
)

# Button widget calls the above function when clicked
apibutton.on_click(api_button_callback)

## Interactive graph

The graph below shows PHE COVID-19 data from 1 January to 30 November 2021 (when this dashboard was created). If this range is now out of date, please press the button below to refresh the dataset, after which the graph should update itself. If there is any issue requesting new data from PHE, the dashboard will fall back to using the original data.

The following metrics are shown in the graph:
* Mechanical ventilator beds occupied by COVID-19 patients (*'beds'*)[<sup>1</sup>](#fn1)
* New cases by publish date (*'cases'*)
* New people who received first vaccination dose by publish date (*'firstDose'*)
* New people who received first vaccination dose by publish date (*'secondDose'*)

As the numbers in the *'beds'* metric are significantly lower than the other three metrics, the data is displayed in a logarithmic scale. This also allows the representation of **exponential growth** within a small table, which be challenging with other forms of graph.

For example, the period of time on the graph immediately after January 2021 shows an exponential rise of first vaccine doses being administered (from zero, effectively). The **y** scale shows around 100,000 first vaccine doses were administered (10<sup>5</sup>), approximately 50,000 new cases were published (10<sup>4</sup>), and approximately 4000 mechnical ventilator beds were occupied by COVID-19 patients (10<sup>3</sup>).

##### <span id="fn1">1. It is assumed that the 'mechanical ventilator beds occupied by COVID patients' metric is cumulative rather than new patients each day, as the Developer's Guide for the API does not provide detail.</span>

In [57]:
bedcols=wdg.SelectMultiple(
    options=['beds', 'cases', 'firstDose', 'secondDose'], # Data options available to add to the graph
    value=['beds', 'cases', 'firstDose'], # Initial value plotted on the graph
    rows=4, # Rows of the selection box
    description='Options',
    disabled=False
)

def refresh_graph():
    # Force redrawing of the graph after changing value in the selection widget
    current=bedcols.value
    if current==bedcols.options[0:2]:
        other=bedcols.options[0:3]
    else:
        other=bedcols.options[0:2]
    bedcols.value=other
    bedcols.value=current

def beds_graph(graphcolumns):
    # Callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        df.plot(logy=True, y=list(graphcolumns)) # Graphcolumns is a tuple - need a list
        plt.title("UK COVID-19: Ventilator Beds, Cases and Vaccinations")
        plt.xlabel("Month")
        plt.ylabel("Number of people")
        plt.show() # Important - graphs won't update properly if this is missing
    else:
        # If the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
# Keep calling beds_graph(graphcolumns=value_of_bedcols); capture output in widget output    
output=wdg.interactive_output(beds_graph, {'graphcolumns': bedcols})

controls=wdg.HBox([bedcols, apibutton])
display(controls, output)

Output()